# Examples for interacting with Database

In [2]:
import utils
from psycopg2.extras import RealDictCursor

### Fetching Movies and Users

In [5]:
movieid = "bigger+stronger+faster*+2008"
userid = 28309

'''
get_connection() uses your information from your config.py, make sure it is updated

TLDR: The stream is taking a long time to catch up to current data so there is not current data
in the database yet. There will be soon. For current data pass database="current" to get_connection()
'''
conn = utils.get_connection() # utils.get_connection(database="current")

'''
movie and user will be dictionaries with the columns of database as features
The fun thing about these commands is they will fetch from the database for you if it exists,
and they will request the api (and add to the database!) if it doesn't exist

...Unless of course you set commit=False, but don't unless you have a specific reason, thats for the stream
'''
movie = utils.fetch_movie(conn, movieid)
user = utils.fetch_user(conn, userid)
print(movie)
print()
print(user)

'''
!!!!! REMEMBER TO CLOSE YOUR CONNECTION !!!!!
I would handle this for you if I could but I cannot...
...not without taking a lot of flexibility anyway
'''
conn.close()

RealDictRow([('id', 'bigger+stronger+faster*+2008'), ('imdb_id', 'tt1151309'), ('tmdb_id', 13636), ('vote_avg', 7.0), ('vote_count', 41), ('title', 'Bigger Stronger Faster*'), ('language', 'en'), ('adult', False), ('genres', ['Documentary']), ('runtime', 105), ('popularity', 6.108182)])

RealDictRow([('userid', 28309), ('age', 33), ('occupation', 'executive/managerial'), ('gender', 'M')])


### Running Custom SQL

In [22]:
# Get a connection
conn = utils.get_connection()

#### Ratings Table

In [23]:
'''
Ratings table has four columns:
datetime
userid
movieid
rating
'''
# Create a cursor, this will allow you to interact with the database
# RealDictCurso will give you return as a dictionary
cur = conn.cursor(cursor_factory=RealDictCursor)

# Getting all ratings
sql = "SELECT * FROM ratings;"
cur.execute(sql)
# Lets grab one from the result and print it
first = cur.fetchone()
print(first)
print(first["datetime"])
print(cur.fetchone())

# Lets print how many we actually fetched, this command will take more time with larger db
print(len(cur.fetchall()))
# Notice that this is two less than is what is in the database
# Thats because we already fetched two! Keep that in mind.

# Now heres an example of a more complicated SQL query that you may use
sql = "SELECT userid, movieid, rating FROM ratings WHERE rating < 3"
cur.execute(sql)
print(cur.fetchone())
print(len(cur.fetchall()))

# Here is an example of getting distinct rows (They should already be distinct buuuuutttttt, can neve be too sure ya know)
sql = "SELECT DISTINCT userid, movieid FROM ratings;"
cur.execute(sql)
print(len(cur.fetchall()))

RealDictRow([('datetime', datetime.datetime(2024, 2, 27, 3, 50, 17)), ('userid', 94224), ('movieid', 'the+lion+king+1994'), ('rating', 4)])
2024-02-27 03:50:17
RealDictRow([('datetime', datetime.datetime(2024, 2, 27, 3, 50, 17)), ('userid', 425733), ('movieid', 'henry+poole+is+here+2008'), ('rating', 4)])
471
RealDictRow([('userid', 283747), ('movieid', 'boxing+helena+1993'), ('rating', 2)])
4
473


#### Views Table

In [24]:
'''
Views has 5 columns:
datetime
userid
movieid
watchtime
movielength

This is a brand new table! It records when users request a part of a film.
The watchtime is how many minutes they have watched this film.
The movielength is how long the film is.
Yes, watchtime can (and will in many cases!) be longer than movielength, this means they rewatched!
...Or I guess watched the same minute multiple times...Or tried multiple time to get through it and cant
Well anyway, recording the exact minute they requested is possible but not clear how to utilize that in a meaningful way so this is what you get. :)
'''
cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "SELECT * FROM views;"

cur.execute(sql)
view = cur.fetchone()
print(view)
print(len(cur.fetchall()))
# This table will have any combination of (userid,movieid) once where the userid requested the movieid, each time after increments watchtime


RealDictRow([('datetime', datetime.datetime(2024, 2, 27, 3, 51, 31)), ('userid', 467414), ('movieid', 'hannibal+rising+2007'), ('watchtime', 1), ('movielength', 121)])
27487


### Recommendations Table

In [25]:
'''
The Recommendations Table has 6 columns:
datetime
userid
server
status
results
responsetime

These are all pretty self explanatory. The main reason to query this table would be in combination with views to see if the user watched something we recommended.
'''

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "SELECT * FROM recommendations;"

cur.execute(sql)
rec = cur.fetchone()
print(rec)
print(len(cur.fetchall()))

RealDictRow([('datetime', datetime.datetime(2024, 2, 27, 3, 33, 48)), ('userid', 404809), ('server', '17645-team21.isri.cmu.edu:8082'), ('status', 200), ('results', ['the+color+purple+1985', 'hollywoodland+2006', 'dr.+horribles+sing-along+blog+2008', 'the+thin+man+1934', 'pusher+iii+im+the+angel+of+death+2005', 'the+wicker+man+1973', 'michael+1996', '28+days+later+2002', 'metro+1997', 'the+man+who+fell+to+earth+1976', 'petes+dragon+1977', 'punch-drunk+love+2002', 'shampoo+1975', 'robot+jox+1989', 'the+daytrippers+1996', 'dazed+and+confused+1993', 'panic+room+2002', 'vampire+hunter+d+bloodlust+2000', 'in+love+and+war+1996', 'where+the+heart+is+2000']), ('responsetime', 41)])
305


In [26]:
'''
!!!!! Don't Forget, Your Computer WILL Explode (Not really, but don't forget) !!!!!
'''
conn.close()

### Querying Monitoring

In [2]:
from prometheus_api_client import PrometheusConnect
from datetime import datetime, timedelta

### Example of how to get even and odd averages

In [7]:
prom_url = "http://128.2.205.122:9090"
prom = PrometheusConnect(url=prom_url)
# Calculate start and end time
end_time = datetime.now().replace(microsecond=0)  # Current time
start_time = end_time - timedelta(days=1)  # One day ago
start_time_str = start_time.isoformat() + 'Z'
end_time_str = end_time.isoformat() + 'Z'
even = prom.custom_query_range(query="histogram_quantile(0.5, sum by(le) (rate(even_rating_bucket[1h])))", start_time=start_time, end_time=end_time, step='1d')
odd = prom.custom_query_range(query="histogram_quantile(0.5, sum by(le) (rate(odd_rating_bucket[1h])))", start_time=start_time, end_time=end_time, step='1d')
print("Even Avg:",float(even[0]['values'][0][1])), print("Odd Avg:",float(odd[0]['values'][0][1]))

Even Avg: 3.3537682789651293
Odd Avg: 3.3286180631120783


(None, None)

In [3]:
# If we have hits this should get them, seperated by model
prom_url = "http://128.2.205.122:9090"
prom = PrometheusConnect(url=prom_url)
# Calculate start and end time
end_time = datetime.now().replace(microsecond=0)  # Current time
start_time = end_time - timedelta(days=1)  # One day ago
start_time_str = start_time.isoformat() + 'Z'
end_time_str = end_time.isoformat() + 'Z'
data = prom.custom_query_range(query="hit_rate_total", start_time=start_time, end_time=end_time, step='1d')
data

[]

## The End
Reach out with questions!